In [1]:
!pip install pandas numpy catboost icecream matplotlib seaborn statsmodels scikit-learn

In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from icecream import ic
import matplotlib
import matplotlib.pyplot as plt
import os

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.stats.diagnostic import het_arch

from sklearn.metrics import root_mean_squared_error, mean_absolute_error
import seaborn as sns


In [4]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
matplotlib.use("Agg")  # disables GUI plotting

In [5]:
dataset_df = pd.read_csv("./data/dataset.csv")
promotions_df = pd.read_csv("./data/promotions.csv")
stores_df = pd.read_csv("./data/stores.csv")


First Question

In [11]:
emptypromosinceweek = len(stores_df['PromoSinceWeek'][stores_df['PromoSinceWeek'].isna()])
emptypromosinceyear = len(stores_df['PromoSinceYear'][stores_df['PromoSinceYear'].isna()])
emptyPromoInterval = len(stores_df['PromoInterval'][stores_df['PromoInterval'].isna()])
emptycomptdistance = len(stores_df['CompetitionDistance'][stores_df['CompetitionDistance'].isna()])

In [13]:
print(emptypromosinceweek)

544


In [15]:
print(emptypromosinceyear)
print(emptyPromoInterval)
print(emptycomptdistance)


544
544
3


In [17]:
# Second Question
result_df_total_sales_customers = dataset_df.groupby('Store')[['Sales', 'Customers']].sum().reset_index()
print(len(result_df_total_sales_customers))
print(len(stores_df))
merged_data = pd.merge(result_df_total_sales_customers, stores_df, on='Store', how='inner')
merged_data = merged_data[['Store', "Sales", "Customers", "Assortment"]].copy()
mean_by_assortment = merged_data.groupby('Assortment')[['Sales', 'Customers']].mean().reset_index()
mean_by_assortment

1115
1115


,Assortment,Sales,Customers
0,Large,5.568195e+06,5.736887e+05
1,Medium,7.882924e+06,1.885836e+06
2,Small,4.967538e+06,5.611584e+05


In [19]:
# Third Question
dataset_df['Date'] = pd.to_datetime(dataset_df['Date'])
dataset_df_2014 = dataset_df[dataset_df['Date'].dt.year == 2014]
dataset_df_2014_sorted = dataset_df_2014.sort_values(by='Sales', ascending=False)
print(dataset_df_2014_sorted.iloc[0]['Store'])

57


In [23]:
# Fourth Question
dataset_df['Date'] = pd.to_datetime(dataset_df['Date'])
stores_df_filtered = stores_df[
    (stores_df['CompetitionDistance'] < 1000) &
    (stores_df['PromoInterval'] == 'Jan,Apr,Jul,Oct')
]
dataset_df_grouped = dataset_df.groupby(["Store", dataset_df['Date'].dt.to_period('M')])['Sales'].mean().reset_index()
merged_data = pd.merge(stores_df_filtered, dataset_df_grouped,  on='Store', how='left')
merged_data = merged_data[['Store', 'Date', 'Sales']].copy()
merged_data

,Store,Date,Sales
0,2,2013-01,4429.653846
1,2,2013-02,4629.750000
2,2,2013-03,5221.000000
3,2,2013-04,4675.120000
4,2,2013-05,4849.125000
...,...,...,...
3195,1104,2015-03,5429.961538
3196,1104,2015-04,5623.875000
3197,1104,2015-05,5520.086957
3198,1104,2015-06,5543.960000


In [25]:
# Define functions for plotting


# **ACF**
# Measures correlation between the series and its lagged versions **including indirect effects**.

# ## **PACF**
# Measures **direct** correlation between the series and its lag at k **after removing effects of all intermediate lags**.
def plot_acf_pacf(df, store_id, max_lag=60):
    df = df[df["empty_store_flag"] == 0]
    s = (df.loc[df["Store"] == store_id, ["Date", "Sales"]].sort_values("Date").set_index("Date")["Sales"])

    s = s.asfreq("D")
    s_clean = s.dropna()
    fig, axes = plt.subplots(1, 2, figsize=(12,4))
    plot_acf(s_clean, lags=max_lag, ax=axes[0])
    plot_pacf(s_clean, lags=max_lag, ax=axes[1], method="ywm")
    axes[0].set_title("ACF")
    axes[1].set_title("PACF")
    fig.savefig(f"acf_pacf_plot_{store_id}.png", dpi=300, bbox_inches="tight")

def plot_sales_hist(df, store_id):
    df = df[df["empty_store_flag"] == 0]
    s = df.loc[df["Store"] == store_id, ["Date", "Sales"]]

    fig, ax = plt.subplots(figsize=(10, 6))
    sns.histplot(s['Sales'], bins=40, kde=False, color='skyblue', ax=ax)
    ax.set_title(f'Sales Distribution - Store {store_id}')
    ax.set_xlabel('Sales')
    ax.set_ylabel('Frequency')

    fig.savefig(f"sales_histogram_{store_id}.png", dpi=300, bbox_inches="tight")
    plt.close(fig)

def plot_seasonal_decompose(df, store_id):
    df = df[df["empty_store_flag"] == 0]
    s = df.loc[df["Store"] == store_id, ["Date", "Sales"]]
    s = s['Sales'].astype(float)

    #s = np.log1p(s['Sales'].astype(float))
    trs = seasonal_decompose(s, model='additive', period=7)

    # Check for heteroskedasticity
    resid = trs.resid
    resid = resid[~resid.isna()]
    test_stat, p_value, _, _ = het_arch(resid)

    # if < 0.05 Residuals are heteroskedastic (ARCH effect exists)
    ic(store_id)
    print("ARCH test statistic:", test_stat)
    print("p-value:", p_value)

    fig = trs.plot()  # capture the figure
    fig.set_size_inches(12, 8)  # optional resize
    fig.tight_layout()
    fig.savefig(f"non_log_seasonal_decompose_{store_id}.png", dpi=300, bbox_inches="tight")
    plt.close(fig)


In [27]:
# Modelling part
# Reload the datasets

In [29]:
dataset_df = pd.read_csv("./data/dataset.csv")
promotions_df = pd.read_csv("./data/promotions.csv")
stores_df = pd.read_csv("./data/stores.csv")

dataset_df['Date'] = pd.to_datetime(dataset_df['Date'])


In [31]:
# Get missing dates for each store
def missing_dates(dates):
    full = pd.date_range(dates.min(), dates.max(), freq='D')
    return full.difference(dates)

missing = dataset_df.groupby('Store')['Date'].apply(missing_dates)
missing

Store
1       DatetimeIndex(['2013-01-06', '2013-01-13', '20...
2       DatetimeIndex(['2013-01-06', '2013-01-13', '20...
3       DatetimeIndex(['2013-01-06', '2013-01-13', '20...
4       DatetimeIndex(['2013-01-06', '2013-01-13', '20...
5       DatetimeIndex(['2013-01-06', '2013-01-13', '20...
                              ...                        
1111    DatetimeIndex(['2013-01-06', '2013-01-13', '20...
1112    DatetimeIndex(['2013-01-06', '2013-01-13', '20...
1113    DatetimeIndex(['2013-01-06', '2013-01-13', '20...
1114    DatetimeIndex(['2013-01-06', '2013-01-13', '20...
1115    DatetimeIndex(['2013-01-06', '2013-01-13', '20...
Name: Date, Length: 1115, dtype: object

In [33]:
# Fill missing dates for each store and create empty store time flag
def fill_dates(dates):
    full_index = pd.date_range(start=dates["Date"].min(), end=dates["Date"].max(), freq='D')
    dates = dates.set_index('Date').reindex(full_index)
    return dates

dataset_df = dataset_df.groupby('Store', group_keys=False).apply(fill_dates).reset_index().rename(columns={'index': 'Date'})
dataset_df['Store'] = dataset_df['Store'].ffill()
dataset_df['empty_store_flag'] = 0
dataset_df.loc[dataset_df['Sales'].isna(), 'empty_store_flag'] = 1
dataset_df["Sales"] = dataset_df["Sales"].fillna(0)
dataset_df["Customers"] = dataset_df["Customers"].fillna(0)

C:\Users\eness\AppData\Local\Temp\ipykernel_53784\3424360171.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dataset_df = dataset_df.groupby('Store', group_keys=False).apply(fill_dates).reset_index().rename(columns={'index': 'Date'})


In [35]:
# Get and merge daily spot promo flag
promotions_df['spot_promo_flag'] = 1
promotions_df['Date'] = pd.to_datetime(promotions_df['Date'])
merged_data = pd.merge(dataset_df, promotions_df, on=['Store', "Date"], how='left')
merged_data['spot_promo_flag'] = merged_data['spot_promo_flag'].fillna(0)
merged_data_all = pd.merge(merged_data, stores_df, on='Store', how = 'left')

In [37]:
# Calculate cyclical promo flag
def check_cyclical_promo(row):
    month_map = {
        'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
        'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
        'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
    }

    if pd.isna(row['PromoSinceYear']) or row['PromoSinceWeek'] == 0:
        return 0

    if pd.isna(row['PromoInterval']):
        return 0

    try:
        promo_start_week = int(row['PromoSinceWeek'])
        promo_start_year = int(row['PromoSinceYear'])
    except:
        return 0

    current_year = row['Date'].year
    current_week = row['Date'].isocalendar()[1]
    current_month = row['Date'].month

    if current_year < promo_start_year:
        return 0
    elif current_year == promo_start_year and current_week < promo_start_week:
        return 0
    else:
        promo_months = row['PromoInterval'].split(',')
        promo_month_nums = [month_map.get(m.strip(), 0) for m in promo_months]

        if current_month in promo_month_nums:
            return 1
        else:
            return 0

merged_data_all['cyclical_promo_flag'] = merged_data_all.apply(check_cyclical_promo, axis=1)

In [38]:
# Preprocess competition distance
merged_data_all['empty_comp_distance'] = 0
merged_data_all.loc[merged_data_all['CompetitionDistance'].isna(), 'empty_comp_distance'] = 1
merged_data_all['CompetitionDistance'] = merged_data_all['CompetitionDistance'].fillna(max(merged_data_all['CompetitionDistance']))
merged_data_all['CompetitionDistance'] = np.log1p(merged_data_all['CompetitionDistance'])


In [41]:

features = merged_data_all[['Store', 'Date', "Customers", 'Sales', 'spot_promo_flag', 'CompetitionDistance',
                            'cyclical_promo_flag', 'empty_comp_distance', "empty_store_flag"]]

# Run plotting utilities and save them
stores = features['Store'].unique()
for s in stores[::180]:
    tmp = features[features['Store'] == s].sort_values('Date')
    tmp_ = tmp[tmp["empty_store_flag"] == 0]
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(tmp_['Date'], tmp_['Sales'])
    ax.set_title(f"Store {s} – Sales Over Time")
    ax.set_xlabel("Date")
    ax.set_ylabel("Sales")

    fig.savefig(f"plot_{s}.png", dpi=300, bbox_inches="tight")
    plot_acf_pacf(features, s, max_lag=60)
    plot_sales_hist(features, s)
    plot_seasonal_decompose(features, s)


ic| store_id: 1.0


ARCH test statistic: 137.49207420472044
p-value: 1.3752703166849213e-24


ic| store_id: 181.0


ARCH test statistic: 95.61905333153963
p-value: 4.088052029681499e-16


ic| store_id: 361.0


...

ARCH test statistic: 50.63220830183033
p-value: 2.041617010257462e-07


ic| store_id: 901.0


ARCH test statistic: 90.6697126378063
p-value: 3.944480448796724e-15


ic| store_id: 1081.0


ARCH test statistic: 52.06154201182207
p-value: 1.1115672232231243e-07


In [43]:
# Run preliminary feature engineering steps
features = features.sort_values(["Store","Date"])

features['assortment_code'] = merged_data_all['Assortment'].astype('category').cat.codes
features['Store'] = features['Store'].astype(str)  

features["weekday"] = features["Date"].dt.weekday
features["is_weekend"] = (features["weekday"] >= 5).astype(int)
features["day"] = features["Date"].dt.day
features["month"] = features["Date"].dt.month
features["year"] = features["Date"].dt.year


In [45]:
# Sales lag features
for L in [1,2,3,4,5,6, 7, 14, 21,  28, 35, 364]:
    features[f"Sales_lag_{L}"] = features.groupby("Store")["Sales"].shift(L)
    features[f"Customer_lag_{L}"] = features.groupby("Store")["Customers"].shift(L)

features["Sales_lag_diff_1"] =  features["Sales_lag_1"] -  features["Sales_lag_7"]
features["Sales_lag_diff_2"] =  features["Sales_lag_7"] -  features["Sales_lag_14"]

features["spot_promo_x_weekend"] = features["spot_promo_flag"] * features["is_weekend"]



In [53]:
# Fourier transform features, fourier features give our model a compact, smooth representation of repeating seasonality.
g = features.groupby("Store")
features["t"] = g["Date"].transform(lambda x: (x - x.min()).dt.days)
for k in [1, 2, 3]:
    features[f"sin_wk_{k}"] = np.sin(2*np.pi *k * features["t"] / 7)
    features[f"cos_wk_{k}"] = np.cos(2*np.pi *k * features["t"] / 7)


In [55]:
# An exponentially weighted (EW) mean is a moving average where recent observations
# get more weight and older observations get exponentially decreasing weight.
features["ewma_7"] = features.groupby("Store")["Sales"].transform(lambda s: s.shift(1).ewm(span=7, adjust=False, min_periods=3).mean())
features["ewma_28"] = features.groupby("Store")["Sales"].transform(lambda s: s.shift(1).ewm(span=28, adjust=False, min_periods=7).mean())

# Exponentially weighted std (volatility), e.g., 7-day
features["ewstd_7"] = features.groupby("Store")["Sales"].transform(lambda s: s.shift(1).ewm(span=7, adjust=False, min_periods=3).std())
# Compute rolling averages and rolling standard deviations over 7-28 days, use at least 3 valid days
for W in [7, 28, 56]:
    features[f"Sales_rollmean_{W}"] = features.groupby("Store")["Sales"].transform(lambda s: s.shift(1).rolling(W, min_periods=3).mean())
    features[f"Sales_rollstd_{W}"] = features.groupby("Store")["Sales"].transform(lambda s: s.shift(1).rolling(W, min_periods=3).std())

# A scale‑free momentum signal. By dividing “yesterday” by a recent average,
# we tell the model whether the latest level is unusually high or low for this store right now,
# without caring about the store’s absolute scale
features["lag1_over_roll7"] = features["Sales_lag_1"] / (features["Sales_rollmean_7"] + 1e-6)


In [51]:
feature_cols = ['Store', 'spot_promo_flag', 'CompetitionDistance', "empty_store_flag",
           'cyclical_promo_flag', 'empty_comp_distance', 'assortment_code', 'weekday',
           'is_weekend', 'day', 'month', 'year', 'Sales_lag_1', 'Customer_lag_1', 'Sales_lag_2',
           'Customer_lag_2', 'Sales_lag_3', 'Customer_lag_3', 'Sales_lag_4',
           'Customer_lag_4', 'Sales_lag_5', 'Customer_lag_5', 'Sales_lag_6',
           'Customer_lag_6', 'Sales_lag_7', 'Customer_lag_7', 'Sales_lag_14',
           'Customer_lag_14', 'Sales_lag_21', 'Customer_lag_21', 'Sales_lag_28',
           'Customer_lag_28', 'Sales_lag_35', 'Customer_lag_35', 'Sales_lag_364',
           'Customer_lag_364', "spot_promo_x_weekend", "ewma_7", "ewma_28", "ewstd_7",
           'Sales_rollmean_7', 'Sales_rollstd_7', 'Sales_rollmean_28',
           'Sales_rollstd_28', 'Sales_rollmean_56', 'Sales_rollstd_56', "Sales_lag_diff_1", "Sales_lag_diff_2",
            'sin_wk_1', 'cos_wk_1', 'sin_wk_2', 'cos_wk_2', 'sin_wk_3', 'cos_wk_3']

In [57]:
# Single train-validation split, optimize and tune the model using this validation split and calculate final metrics on another test split  
cat_cols = ["Store","assortment_code"]
cat_idxs = [feature_cols.index(c) for c in cat_cols if c in feature_cols]

cutoff = features["Date"].quantile(0.9) # or a specific date
train = features[(features["Date"] <= cutoff)].dropna(subset=feature_cols + ["Sales"])
valid = features[(features["Date"] > cutoff)].dropna(subset=feature_cols + ["Sales"])

X_tr, y_tr = train[feature_cols], train["Sales"]
X_va, y_va = valid[feature_cols], valid["Sales"]


In [59]:
# Optinially, log transform stabilizes variance, reduces heteroskedasticity
# Convert multiplicative effects to additive
y_tr = np.log1p(y_tr)
y_va = np.log1p(y_va)

ic(len(y_tr))
ic(len(y_va))

pool_tr = Pool(X_tr, y_tr, cat_features=cat_idxs)
pool_va = Pool(X_va, y_va, cat_features=cat_idxs)


ic| len(y_tr): 538079
ic| len(y_va): 104773


In [61]:

model = CatBoostRegressor(
    loss_function="RMSE",
    iterations=7000,
    learning_rate=0.04,
    depth=8,
    l2_leaf_reg=3.0,
    subsample=0.8,
    random_seed=42,
    od_type="Iter",
    bootstrap_type='Bernoulli',
    od_wait=200,
    eval_metric="RMSE",
    task_type="CPU",
    verbose=200
    )
model.fit(pool_tr, eval_set=pool_va)
predictions_validation = model.predict(pool_va)


0:	learn: 3.5098904	test: 3.2450109	best: 3.2450109 (0)	total: 479ms	remaining: 55m 54s
200:	learn: 0.1138996	test: 0.1176818	best: 0.1176818 (200)	total: 1m 7s	remaining: 37m 53s
400:	learn: 0.1006818	test: 0.1125498	best: 0.1125498 (400)	total: 2m 12s	remaining: 36m 22s
600:	learn: 0.0948425	test: 0.1109749	best: 0.1109496 (591)	total: 3m 20s	remaining: 35m 31s
800:	learn: 0.0911643	test: 0.1102368	best: 0.1102218 (788)	total: 4m 25s	remaining: 34m 14s
1000:	learn: 0.0885782	test: 0.1097005	best: 0.1096950 (996)	total: 5m 31s	remaining: 33m 7s
1200:	learn: 0.0865641	test: 0.1091737	best: 0.1091669 (1196)	total: 6m 36s	remaining: 31m 53s
1400:	learn: 0.0849646	test: 0.1089549	best: 0.1089549 (1400)	total: 7m 41s	remaining: 30m 44s
1600:	learn: 0.0835268	test: 0.1086217	best: 0.1086191 (1599)	total: 8m 46s	remaining: 29m 34s
1800:	learn: 0.0823253	test: 0.1083694	best: 0.1083405 (1791)	total: 9m 51s	remaining: 28m 26s
2000:	learn: 0.0813122	test: 0.1080245	best: 0.1080245 (2000)	total:

In [62]:

# Feature importance, this takes too long
#    importance = model.get_feature_importance(data=pool_tr, type="LossFunctionChange")
#    fi = pd.DataFrame({
#        'feature': model.feature_names_,
#        'importance': importance
#    }).sort_values('importance', ascending=False)


In [63]:
# Calculate metrics
yhat = np.expm1(predictions_validation)
y_og = np.expm1(y_va)
valid['predictions'] = yhat

model_rmse_error = root_mean_squared_error(yhat, y_og)
ic(model_rmse_error)

# MAE is ideal when we want a straightforward measure of average error magnitude without emphasizing large errors disproportionately.
model_mae_error = mean_absolute_error(yhat, y_og)
ic(model_mae_error)


ic| model_rmse_error: 761.7067372905884
ic| model_mae_error: 481.74724807668065


481.74724807668065

In [64]:
# Per Store Error
stores = features['Store'].unique()
for store_id in stores[::180]:
    yhatfilter = valid[valid['Store'] == str(store_id)]["predictions"]
    yogfilter =  valid[valid['Store'] == str(store_id)]["Sales"]
    model_rmse_error = root_mean_squared_error(yhatfilter, yogfilter)
    ic(store_id)
    ic(model_rmse_error)


ic| store_id: '1.0'
ic| model_rmse_error: 370.0122108467618
ic| store_id: '181.0'
ic| model_rmse_error: 491.5619884700495
ic| store_id: '361.0'
ic| model_rmse_error: 629.2889803451918
ic| store_id: '541.0'
ic| model_rmse_error: 721.8251357376301
ic| store_id: '721.0'
ic| model_rmse_error: 693.3851971638824
ic| store_id: '901.0'
ic| model_rmse_error: 641.9562532150703
ic| store_id: '1081.0'
ic| model_rmse_error: 959.3812235499787


In [65]:
# Naive baselines to compare, just output 7-1-14 days lagged sales value
naive_rmse_error_lag7 = root_mean_squared_error(X_va['Sales_lag_7'], y_og)
ic(naive_rmse_error_lag7)

naive_mae_error_lag7 = mean_absolute_error(X_va['Sales_lag_7'], y_og)
ic(naive_mae_error_lag7)

naive_rmse_error_lag1 = root_mean_squared_error(X_va['Sales_lag_1'], y_og)
ic(naive_rmse_error_lag1)

naive_mae_error_lag1 = mean_absolute_error(X_va['Sales_lag_1'], y_og)
ic(naive_mae_error_lag1)

naive_rmse_error_lag14 = root_mean_squared_error(X_va['Sales_lag_14'], y_og)
ic(naive_rmse_error_lag14)

naive_mae_error_lag14 = mean_absolute_error(X_va['Sales_lag_14'], y_og)
ic(naive_mae_error_lag14)



ic| naive_rmse_error_lag7: 3266.755906463662
ic| naive_mae_error_lag7: 2175.0095062659275
ic| naive_rmse_error_lag1: 4785.5996951762345
ic| naive_mae_error_lag1: 3100.397631069073
ic| naive_rmse_error_lag14: 2594.4611624330373
ic| naive_mae_error_lag14: 1425.2945128993156


1425.2945128993156

In [91]:
# Optinionally run random search hyperparameter optimization, we may setup a Bayesian hyperparameter optimization loop using ray tune library. 
# This is especially useful when multiple gpus are available  

In [75]:
def sample_params(rng):
    return {
        "depth": int(rng.integers(6, 11)), # 6..10
        "learning_rate": float(rng.uniform(0.01, 0.08)), 
        "bagging_temperature": float(rng.uniform(0.0, 1.0)),
        "subsample": float(rng.uniform(0.6, 1.0)),
        "colsample_bylevel": float(rng.uniform(0.6, 1.0)),
        "grow_policy": rng.choice(["SymmetricTree", "Lossguide"]),
        "random_strength": float(rng.uniform(0.0, 2.0)),
        }

In [ ]:
n_iter = 10
seed = 25
best = {"rmse": np.inf, "params": None, "cv_scores": None}

for it in range(n_iter):
    rng = np.random.default_rng(seed)
    params = sample_params(rng)
    fold_rmses = []

    model = CatBoostRegressor(
        loss_function="RMSE",
        learning_rate=params["learning_rate"],
        depth=params["depth"],
        bagging_temperature=params["bagging_temperature"],
        subsample=params["subsample"],
        colsample_bylevel=params["colsample_bylevel"],
        grow_policy=params["grow_policy"],
        random_strength=params["random_strength"],
        random_seed=seed,
        od_type="Iter",
        od_wait=200,
        use_best_model=True,
        task_type="CPU",
        verbose=False
    )

    model.fit(pool_tr, eval_set=pool_va, verbose=False)

    # predict on validation, back-transform to original scale
    yhat_log = model.predict(pool_va)
    yhat = np.expm1(yhat_log)
    y_va_og = np.expm1(y_va)
    fold_rmses.append(root_mean_squared_error(y_va_og, yhat))

    avg_rmse = float(np.mean(fold_rmses))
    trials.append({"iter": it+1, "params": params, "cv_rmse": avg_rmse, "folds": fold_rmses})

    if avg_rmse < best["rmse"]:
        best = {"rmse": avg_rmse, "params": params, "cv_scores": fold_rmses}

    print(f"[{it+1}/{n_iter}] RMSE={avg_rmse:.2f} best={best['rmse']:.2f} params={params}")
